# Unit Tests for the Landlab GravelBedrockEroder Component

*G.E. Tucker, CIRES and Department of Geological Sciences, University of Colorado Boulder*

This notebook describes a series of basic unit tests for the `GravelBedrockEroder` component. These tests are implemented in the file `test_gravel_bedrock_eroder.py`, and are in addition to the unit tests implemented as doctests in the source code.

## Types of test

The theory starts with a representation of a layer of sediment overlying rock. Each grid cell is assumed to contain a primary channel that drains to an adjacent cell. The cell may also receive inflow of water and sediment from neighboring cells.

Processes and effects include:

1. Dynamic adjustment of channel width, based on near-threshold theory (implicit in derivation; not calculated explicitly unless requested via function call)
2. Transport of coarse (assumed gravel-size) sediment as bed load
3. Abrasion of sediment, which turns coarse sediment into wash load (not tracked)
4. Abrasion of underlying bedrock
5. Plucking erosion of underlying bedrock, which supplies coarse sediment

Ideally, each of these elements should be tested, both separately and in combination. Two types of test are used: instantaneous tests, which are "single iteration" comparisons between predicted and computed values, and equilibrium tests, in which a small terrain is run with baselevel forcing ("uplift") until it reaches an equilibrium that is then compared with an independently calculated solution.

In addition, the tests should collectively vary each input parameter from its default value at least once. Input parameters and their defaults include:

- intermittency_factor=0.01
- transport_coefficient=0.041
- abrasion_coefficient=0.0
- sediment_porosity=0.35
- depth_decay_scale=1.0
- plucking_coefficient=1.0e-6
- coarse_fraction_from_plucking=1.0

Channel width is adequately tested by the doctests, so additional tests of width are not included here.

## Test setup

The doctests in the code use a `RasterModelGrid`. For these external tests, we will use a `HexModelGrid` with three core nodes and a single open boundary node. This configuration is small enough for tests to be quick and efficient, but large enough to include flow convergence (two cells feel flow into a third, which then drains to the open boundary).

## Imports

In [ ]:
import numpy as np
from numpy.testing import assert_almost_equal
from landlab import HexModelGrid
from landlab.components import FlowAccumulator, GravelBedrockEroder

## Instantaneous tests

### Transport rate

**Test condition**: 3-core-node hex grid with gradient of 0.01 along flow paths. Sediment cover: ample (100 m), limited (0.5 m, with depth decay scale also set to 0.5 m), and none.

Predicted sediment transport rate under ample cover (not limited by bedrock exposure):

$$Q_s = k_Q I Q S^{7/6}$$

Here the default value $k_Q=0.041$ is used, but the intermittency factor is set to 0.02. The discharge by default will be one meter per year times drainage area. The drainage area of one cell is the cell's area, here about 866,025 m$^2$, and the drainage area of the cell that receives flow is three times this. Therefore the predicted sediment transport rate for the two "upstream" cells and for the single "downstream" cell is:

In [ ]:
Q = (3.0**0.5 / 2.0) * 1e6 * np.array([3, 1, 1])
Qs = 0.041 * 0.02 * Q * (0.01)**(7. / 6.)
print('Predicted transport rate:', Qs)

For the case with limiting sediment cover, when the cover thickness is equal to the depth decay scale (set to 0.5 m), the transport rate should be reduced by a factor of $1 - 1/e$. This works out to:

In [ ]:
print('Predicted transport rate:', Qs * (1.0 - np.exp(-1.)))

Finally, with no sediment at all, the transport rate should be zero.

Note that in order to give the grid nodes a gradient of 0.01, the elevations need to rise in the y-direction at a rate equal to 0.01 / cos 30$^\circ$.

In [ ]:
def test_transport_rate():

    grid = HexModelGrid((4, 2), spacing=1000.0)
    grid.status_at_node[grid.perimeter_nodes] = grid.BC_NODE_IS_CLOSED
    grid.status_at_node[0] = grid.BC_NODE_IS_FIXED_VALUE

    elev = grid.add_zeros('topographic__elevation', at='node')
    elev[:] = (0.01 * grid.y_of_node) / np.cos(np.radians(30.0))
    sed = grid.add_zeros('soil__depth', at='node')
    sed[:] = 100.0
    
    fa = FlowAccumulator(grid)
    fa.run_one_step()
    gbe = GravelBedrockEroder(grid, intermittency_factor=0.02, depth_decay_scale=0.5)
    qs_out = grid.at_node['bedload_sediment__volume_outflux']

    gbe.run_one_step(0.0)  # using dt=0 prevents change to sed, rock, or elev
    assert_almost_equal(qs_out[grid.core_nodes], [ 9.88854526,   3.29618175,  3.29618175])

    elev[:] = (0.01 * grid.y_of_node) / np.cos(np.radians(30.0))
    sed[:] = 0.5
    
    gbe.run_one_step(0.0)
    assert_almost_equal(qs_out[grid.core_nodes], [ 6.25075275,  2.08358425,  2.08358425])
    
    elev[:] = (0.01 * grid.y_of_node) / np.cos(np.radians(30.0))
    sed[:] = 0.0
    
    gbe.run_one_step(0.0)
    assert_almost_equal(qs_out[grid.core_nodes], [ 0., 0., 0.])
    

In [ ]:
test_transport_rate()

### Sediment abrasion rate

Consider the first of the cases above, in which the transport rate is about 3.3 m$^3$/y for the upstream cells and 9.9 m$^3$/y for the downstream ones. If the abrasion coefficient is 10$^{-4}$ m$^{-1}$, then we can calculate the resulting lowering rate of the thickness of sediment as:

$$\frac{dH}{dt} = -\beta (Q_{in} + Q_{out}) \lambda / 2 \Lambda$$

where $\beta$ is the abrasion coefficient, $Q_s$ is the transport rate (m$^3$/y), $\lambda$ is the length of the flow path (distance from the node to its downstream neighbor), and $\Lambda$ is the surface area of the cell.

In this case, the numbers are as follows (here sediment flux is half the above case because we are using the default intermittency factor):

In [ ]:
beta = 1.0e-4  # abrasion coefficient, 1/m
Qout = 0.5 * 3.29618175  # transport rate, m3/y
path_length = 1000.0  # node spacing, m
cell_area = 1000.0 * 1000.0 * 0.5 * 3.0**0.5
print('Rate of thickness loss from sediment abrasion (upstream):',
      beta * 0.5 * (0.0 + Qout) * path_length / cell_area
     )

Qin = 2 * Qout
Qout = 0.5 * 9.88854526
print('Rate of thickness loss from sediment abrasion (downstream):',
      beta * 0.5 * (Qin + Qout) * path_length / cell_area
     )

In [ ]:
def test_sediment_abrasion_rate():

    grid = HexModelGrid((4, 2), spacing=1000.0)
    grid.status_at_node[grid.perimeter_nodes] = grid.BC_NODE_IS_CLOSED
    grid.status_at_node[0] = grid.BC_NODE_IS_FIXED_VALUE

    elev = grid.add_zeros('topographic__elevation', at='node')
    elev[:] = (0.01 * grid.y_of_node) / np.cos(np.radians(30.0))
    sed = grid.add_zeros('soil__depth', at='node')
    sed[:] = 100.0
    
    fa = FlowAccumulator(grid)
    fa.run_one_step()
    gbe = GravelBedrockEroder(grid, abrasion_coefficient=1.0e-4)
    gbe.run_one_step(1.0)

    assert_almost_equal(
        gbe._abrasion[grid.core_nodes],
        [4.7576285545378313e-07, 9.515257103302159e-08, 9.515257103302159e-08]
    )

In [ ]:
test_sediment_abrasion_rate()

### Bedrock abrasion

Here we test the calculation of bedrock abrasion rate. We need a test that has some sediment, but not so much that the bed is totally shielded. We'll use 1 m thick sediment. That reduces the transport capacity, which should be equal to the above transport rates times the fractional alluvial cover, which is 1 minus the bedrock exposure fraction:

In [ ]:
beta = 1.0e-4
path_length = 1000.0
frac_bed_exposed = np.exp(-1.0)
cell_area = 1.0e6 * 0.5 * 3.0**0.5
Qs_out = (
    0.041
    * 0.01
    * 0.01**(7./6.)
    * cell_area
    * np.array([3, 1, 1])
    * (1.0 - frac_bed_exposed)
)
Qs_in = np.array([Qs_out[1] + Qs_out[2], 0., 0.])

print('Sed outflux:', Qs_out)
print('Sed influx:', Qs_in)

sed_abr_rate = beta * 0.5 * (Qs_in + Qs_out) * path_length / cell_area
print('Sediment abrasion rate:', sed_abr_rate)

rock_abr_rate = sed_abr_rate * frac_bed_exposed
print('Bed abrasion rate:', rock_abr_rate)

In [ ]:
def test_rock_abrasion_rate():

    grid = HexModelGrid((4, 2), spacing=1000.0)
    grid.status_at_node[grid.perimeter_nodes] = grid.BC_NODE_IS_CLOSED
    grid.status_at_node[0] = grid.BC_NODE_IS_FIXED_VALUE

    elev = grid.add_zeros('topographic__elevation', at='node')
    elev[:] = (0.01 * grid.y_of_node) / np.cos(np.radians(30.0))
    sed = grid.add_zeros('soil__depth', at='node')
    sed[:] = 1.0
    
    fa = FlowAccumulator(grid)
    fa.run_one_step()
    gbe = GravelBedrockEroder(grid, abrasion_coefficient=1.0e-4)
    gbe.run_one_step(1.0)

    assert_almost_equal(
        gbe._sediment_outflux[grid.core_nodes],
        [ 3.12537638,  1.04179213,  1.04179213]
    )
    assert_almost_equal(
        gbe._rock_abrasion_rate[grid.core_nodes],
        [1.10635873e-07,   2.21271745e-08,   2.21271745e-08]
    )

In [ ]:
test_rock_abrasion_rate()

### Test plucking erosion

Here we can use test cases with zero, thin, and thick sedimentary covers.

The theory behind the plucking rate goes something like this. Assume that the bankfull plucking rate is proportional to excess shear stress to a power. Whipple et al. (2000) suggested that the block loosening rate would be proportional to bed-load transport rate, which in turn tends to scale with excess stress to the 3/2 power. They also suggested that the block extraction rate might also scale linearly to weakly nonlinearly with excess shear stress. Here we take 3/2 as a suitable choice for "weakly nonlinear", and assume that the combination of block loosening and block extraction results in a rate of bedrock lowering by plucking (at bankfull stage) that scales with $(\tau - \tau_c)^{3/2}$. Given near-threshold theory, this then produces the same scaling as the bed-load transport rate, with an unknown coefficient:

$$k_p I Q S^{7/6} e^{-H/H_s}$$

where the exponential factor represents the fractional bedrock exposure.

Now this is the rate of lowering per unit bed area. To convert this into an average rate of lowering across a grid cell, one needs to make a choice. Option 1 is to assume that the portions of the cell outside of the channel "follow the channel down" at the same rate, by some process of rapid near-channel hillslope response. The rate of volume removal of rock per time across the cell is then the incision rate times the cell area:

$$V_p = k_p I Q S^{7/6} e^{-H/H_s} \Lambda$$

where $\Lambda$ is grid-cell surface area. The resulting lowering rate is simply this volume rate divided by cell area:

$$E_p = k_p I Q S^{7/6} e^{-H/H_s}$$

In this case, the rate of incision does not depend on grid-cell size, but the rate of volumetric rate of rock erosion in the cell does (the wider the cell is relative to the channel, the greater the extra area that is effectively controlled by the embedded channel).

Option 2 is to calculate the *average* rate of lowering across the grid cell, by combining the area of the cell that is undergoing channel downcutting and the area that is not. In this case, the volumetric rock erosion rate is the incision rate per unit bed area times the surface area of the channel, which can be approximated as the width of a channel exiting the cell times its length, $\lambda$:

$$V_p = k_p I Q S^{7/6} e^{-H/H_s} \lambda b$$

and the average lowering rate across the cell is:

$$E_p = k_p I Q S^{7/6} e^{-H/H_s} \lambda b / \Lambda$$

For a square cell of width $\Delta x$, this becomes:

$$E_p = k_p I Q S^{7/6} e^{-H/H_s} b / \Delta x$$

Here's an aside, a kind of thought experiment. Suppose you wanted to reduce this cell-size dependence. Imagine you tried assigning some fraction of the erosion at the cell to some of its neighbors. Suppose for the sake of argument that you did this by considering the probability that a portion of the channel laps onto a neighboring cell. One way to think about it is as an average overlap. If a channel is 1 m wide in a 5 m wide cell and you move it from side to side so that its centerline goes from right up against one side to right up against the other, you have an overlap function (overlap length as a function of position) that looks like this: `\___/`. Within the sloping bits, which make up b/dx of the domain, the average overlap should be b/4 (b/2 is the max, 0 is the min, and it's a linear function). So the weighted average is $((b/dx) (b/4) + ((dx-b)/dx) 0 = b^2 / 4 dx$. If $b=0$, the overlap function is of course zero. If $b=dx$, the overlap function is $dx/4$, meaning perhaps that 3/4 of the calculated erosion happens in the cell itself, and the rest is distributed among neighbors somehow. This treatment has the result of causing the $\Delta x$ in the denominator to go away, which seems promising...


For the zero-sediment case, we'll change the default plucking rate coefficient to 10x higher, at $k_p = 10^{-3}$.

## References and further reading

Attal, M., & Lavé, J. (2006). Changes of bedload characteristics along the Marsyandi River (central Nepal): Implications for understanding hillslope sediment supply, sediment load evolution along fluvial networks, and denudation in active orogenic belts. Geol. Soc. Am. Spec. Pap, 398, 143-171.

Attal, M., & Lavé, J. (2009). Pebble abrasion during fluvial transport: Experimental results and implications for the evolution of the sediment load along rivers. Journal of Geophysical Research: Earth Surface, 114(F4).

Grant, G. E. (1997). Critical flow constrains flow hydraulics in mobile‐bed streams: A new hypothesis. Water Resources Research, 33(2), 349-358.

Meyer-Peter, E., & Müller, R. (1948). Formulas for bed-load transport. In IAHSR 2nd meeting, Stockholm, appendix 2. IAHR.

Parker, G. (1978). Self-formed straight rivers with equilibrium banks and mobile bed. Part 2. The gravel river. Journal of Fluid mechanics, 89(1), 127-146.

Phillips, C. B., & Jerolmack, D. J. (2016). Self-organization of river channels as a critical filter on climate signals. Science, 352(6286), 694-697.

Wickert, A. D., & Schildgen, T. F. (2019). Long-profile evolution of transport-limited gravel-bed rivers. Earth Surface Dynamics, 7(1), 17-43.

Willgoose, G., Bras, R. L., & Rodriguez‐Iturbe, I. (1991). A physical explanation of an observed link area‐slope relationship. Water Resources Research, 27(7), 1697-1702.

Willgoose, G. (1994). A physical explanation for an observed area‐slope‐elevation relationship for catchments with declining relief. Water Resources Research, 30(2), 151-159.

Wong, M., & Parker, G. (2006). Reanalysis and correction of bed-load relation of Meyer-Peter and Müller using their own database. Journal of Hydraulic Engineering, 132(11), 1159-1168.